# Query tests

In [1]:
[global]
from dsc.query_engine import Query_Processor as qp
import pandas as pd

def test_syntax(obj, *args):
    try:
        res = obj(*args)
        return res
    except Exception as e:
        print(e)
        assert type(e).__name__ in ['DBError', 'FormatError']
        return None

def test_outcome(obj, fn, verbose, *args):
    res = test_syntax(obj, *args)
    if res is None:
        return
    if verbose:
        for query in res.get_queries():
            print(query)
    writer = pd.ExcelWriter(fn)
    res.output_table.to_excel(writer, 'Sheet1', index = False)
    if len(res.output_tables) > 1:
        for table in res.output_tables:
            res.output_tables[table].to_excel(writer, table, index = False)
    writer.save()    

ash_db = 'data/dsc_result.db'

In [2]:
[syntax]
parameter: db = ''
parameter: t = ''
parameter: c = []
parameter: g = []
t = t.split()
res = test_syntax(qp, db, t, c, g)

In [3]:
[outcome]
parameter: db = ''
parameter: t = ''
parameter: c = []
parameter: g = []
parameter: verbose = 0
output: "/tmp/1.xlsx"
t = t.split()
test_outcome(qp, str(_output), verbose, db, t, c, g)

In [4]:
%sosrun default:1
[1]
# F: undefined module or group name
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist shu.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

Cannot find module ``shu`` in DSC results ``data/dsc_result.db``.


In [5]:
%sosrun default:2
[2]
# F: field name is not pipeline variable
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist simulate.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

Cannot find variable ``mse`` in module ``simulate``


In [6]:
%sosrun default:3 -s force
[3]
# P: overlapping groups
sos_run("outcome", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist tmp.mse score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'],
        g = ["tmp: score_beta, score_pi0", "score: score_beta, score_pi0"])

In [7]:
%sosrun default:4
[4]
# P: use built-in group names
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

In [8]:
%sosrun default:5
[5]
# F: empty table returned
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp < 20', 'shrink.mixcompdist = "normal"'],
        g = ["score: score_beta, score_pi0"])

No results found for targets ``simulate.nsamp, shrink.mixcompdist, score.mse`` under condition ``(simulate.nsamp < 20) AND (shrink.mixcompdist = "normal")``


In [9]:
%sosrun query_show -s force
[query_show]
# P: display query.
sos_run("outcome", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'],
        verbose = 1)

SELECT simulate.nsamp AS simulate_nsamp, shrink.mixcompdist AS shrink_mixcompdist, score_beta.__output__ AS score_beta___output___mse FROM score_beta INNER JOIN shrink ON score_beta.__parent__ = shrink.__id__ INNER JOIN simulate ON shrink.__parent__ = simulate.__id__ WHERE ((simulate.nsamp > 20) AND (shrink.mixcompdist == "normal"))
SELECT simulate.nsamp AS simulate_nsamp, shrink.mixcompdist AS shrink_mixcompdist, score_pi0.__output__ AS score_pi0___output___mse FROM score_pi0 INNER JOIN shrink ON score_pi0.__parent__ = shrink.__id__ INNER JOIN simulate ON shrink.__parent__ = simulate.__id__ WHERE ((simulate.nsamp > 20) AND (shrink.mixcompdist == "normal"))


In [10]:
%sosrun default:6 -s force
[6]
# P: Handle empty field input: return the file path
sos_run("outcome", db = ash_db,
        t = 'simulate shrink score',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])